In [3]:
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import itertools
import csv , codecs
import glob
from PIL import Image
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline



In [4]:
data_dir = "./EEG_data/"

## CSV 파일 생성 함수


### task 0 1 2 : 1-back,  2-back,  rest

### eye 0  1    : close eye,     open eye

### respiratory 0 1 2 : mouse, nose, O2 



In [5]:
def dataset_csv_3():
    normalize_power = str(input("Relative, Abs :"))
    excel_dir = "./EEG_data" + '/' + normalize_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    subject = int(input("subject :"))
    book = Workbook()
    sheet = book.active
    sheet.cell(row=1, column=161).value = 'target'
    sheet.cell(row=1, column=162).value = 'task'
 
    # x축 생성   
    for i in range(1,33):
        sheet.cell(row=1, column=i).value = 'ACh' + str(i)
        sheet.cell(row=1, column=i+32).value = 'BCh' + str(i)
        sheet.cell(row=1, column=i+64).value = 'DCh' + str(i)
        sheet.cell(row=1, column=i+96).value = 'GCh' + str(i)
        sheet.cell(row=1, column=i+128).value = 'TCh' + str(i)
        
    # 파일을 순차적으로 열어서 셀 영역을 복사
    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=20*k+l,column=32*j+m-1).value = source.cell(row=l,column=m).value
        

    with open('./dataset_eeg_3.csv', 'w', newline="") as f:
        c = csv.writer(f)
        for r in sheet.rows:
            c.writerow([cell.value for cell in r])
            
    df = pd.read_csv('./dataset_eeg_3.csv')
    df[df =='                      NaN'] =np.nan
    df = df.fillna(0.0).astype('float64')
    pd.options.mode.chained_assignment = None
#1-back mouse, nose, o2
    df["target"][0*subject:1*subject] = "0" #0:20
    df["target"][1*subject:2*subject] = "1" #21:40
    df["target"][2*subject:3*subject] = "2" #41:60
#2-back mouse, nose, o2
    df["target"][3*subject:4*subject] = "0" #60:80
    df["target"][4*subject:5*subject] = "1" #80:100
    df["target"][5*subject:6*subject] = "2" #100:120
#rest close eye mouse, nose, o2
    df["target"][6*subject:7*subject] = "0" #120:140
    df["target"][7*subject:8*subject] = "1" #140:160
    df["target"][8*subject:9*subject] = "2" #160:180
#rest open eye mouse, nose, o2
    df["target"][9*subject:10*subject] = "0" #180:200
    df["target"][10*subject:11*subject] = "1" #200:220
    df["target"][11*subject:12*subject] = "2" #220:240

    #1-back mouse, nose, o2
    df["task"][0*subject:1*subject] = "1back_m" #0:20
    df["task"][1*subject:2*subject] = "1back_n" #21:40
    df["task"][2*subject:3*subject] = "1back_O2" #41:60
    #2-back mouse, nose, o2
    df["task"][3*subject:4*subject] = "2back_m" #60:80
    df["task"][4*subject:5*subject] = "2back_n" #80:100
    df["task"][5*subject:6*subject] = "2back_O2" #100:120
    #rest close eye mouse, nose, o2
    df["task"][6*subject:7*subject] = "rest_close_m" #120:140
    df["task"][7*subject:8*subject] = "rest_close_n" #140:160
    df["task"][8*subject:9*subject] = "rest_close_O2" #160:180
    #rest open eye mouse, nose, o2
    df["task"][9*subject:10*subject] = "rest_open_m" #180:200
    df["task"][10*subject:11*subject] = "rest_open_n" #200:220
    df["task"][11*subject:12*subject] = "rest_open_O2" #220:240
    
    
    df.to_csv('./dataset_eeg_3.csv')
    
    return df
            


In [6]:
eeg_data_3 = dataset_csv_3()
eeg_data_3

Relative, Abs : Relative
subject : 20


C:\Users\whbom\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32,target,task
0,4.512153,2.236848,16.237565,8.485316,23.868040,8.095887,61.092520,63.790479,68.686679,69.925212,...,0.828638,1.010004,1.778969,6.435780,14.978180,11.507832,11.162954,9.491918,0,1back_m
1,4.539769,4.310226,4.959007,3.965357,7.222414,5.466902,16.751086,19.945360,30.425709,24.128985,...,9.741533,7.106003,10.521237,8.559514,18.069939,15.457455,22.764836,22.393518,0,1back_m
2,9.876811,9.070304,3.572186,4.272411,12.559673,19.156709,26.778865,17.134912,53.187562,49.574373,...,6.724140,8.721257,10.193054,2.678986,11.522934,11.118133,11.282202,10.760432,0,1back_m
3,4.224107,6.190282,8.670873,8.869054,25.241644,20.749929,40.968150,43.190059,42.509827,41.379676,...,4.743475,5.994422,2.769252,5.845854,11.149340,12.461344,12.748985,11.845096,0,1back_m
4,18.441049,19.286110,8.901129,12.888736,7.272771,15.879294,23.182914,38.046045,29.140603,32.524267,...,16.428830,15.406232,16.088470,17.454960,18.750681,17.095190,13.764769,13.443134,0,1back_m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,9.377716,6.913374,5.995995,4.562073,6.141009,6.275420,17.190073,19.530794,29.798448,27.170256,...,11.865520,10.328138,10.171290,11.796915,15.583215,16.539841,15.194092,14.108681,2,rest_open_O2
236,11.906533,10.057115,11.180124,11.478218,14.621087,16.359638,26.560596,24.297633,30.679741,26.798845,...,6.548399,6.806564,10.569691,14.455136,15.406702,16.142647,14.712165,14.551317,2,rest_open_O2
237,10.335814,8.589677,13.796878,13.048695,25.629270,24.416874,33.204969,34.318945,29.184918,28.485295,...,19.061720,18.617480,16.023406,20.075270,26.804320,25.657373,24.098649,19.401579,2,rest_open_O2
238,14.781336,13.040448,5.557476,4.101700,19.404795,10.489655,27.480708,24.014191,32.643752,30.794355,...,9.054677,14.631369,11.264431,17.292341,19.074393,22.937862,23.429094,12.989213,2,rest_open_O2


In [239]:
state={}

state["1back_mouse"]=eeg_data_3.iloc[0:20]
state["1back_nose"]=eeg_data_3.iloc[20:40]
state["1back_O2"]=eeg_data_3.iloc[40:60]

state["2back_mouse"]=eeg_data_3.iloc[60:80]
state["2back_nose"]=eeg_data_3.iloc[80:100]
state["2back_O2"]=eeg_data_3.iloc[100:120]

state["close_rest_mouse"]=eeg_data_3.iloc[120:140]
state["close_rest_nose"]=eeg_data_3.iloc[140:160]
state["close_rest_O2"]=eeg_data_3.iloc[160:180]

state["open_rest_mouse"]=eeg_data_3.iloc[180:200]
state["open_rest_nose"]=eeg_data_3.iloc[200:220]
state["open_rest_O2"]=eeg_data_3.iloc[220:240]

In [240]:
ds_train = {}
ds_test = {}


ds_train={"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
       
       
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
       "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
       "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])
       
       
      }

ds_test = {"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
        
        
               
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
        "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
        "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])

       }




In [241]:
def test(tasks, k_parameter, num_iter):
    result = []
    grid_search= []
    gamma = np.arange(2**(-5),2**(13),2**(0)).tolist()
    C =np.arange(2**(-15),2**(3),2**(0)).tolist()
    
    
    estimator = SVC()
    param_grid = {'kernel':['rbf'], 'gamma': gamma,'C': C }
    grid = GridSearchCV(estimator, param_grid=param_grid)
    
    task = ds_train[tasks]
    X = np.array(task.iloc[:,0:-2])
    Y_idx = np.array(task['task'])
    Y = np.array(task['target'])


    lda = LinearDiscriminantAnalysis(n_components=1)
    lda.fit(X,Y)
    lda_transform = lda.transform(X)

    grid.fit(lda_transform,Y)
    grid_fit_result = pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_score')
    grid_search.append(grid_fit_result)


#         print(grid.best_score_)

#     print('svm best parameters')
#     print(grid.best_params_)

    all_scores = []
    record = [] 
    
    for idx in range(num_iter):
        skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
        skf.get_n_splits(X,Y)
        for train_index, test_index in skf.split(X,Y_idx):
            print(Y_idx[test_index])
            
            
            
            X_final = lda.transform(X)
            svm = SVC(kernel='rbf', C=grid.best_params_['C'], gamma=grid.best_params_['gamma'])
            svm.fit(X_final[train_index], Y[train_index])
            list = [ svm.predict(X_final[test_index]),Y[test_index], Y_idx[test_index] ]
            record.append(list)
            
            
            val_mae = svm.score(X_final[test_index],Y[test_index])
            all_scores.append(val_mae)
#     print('model_score:{}'.format(np.mean(all_scores)))
   
    
    record_result = []
    for j in range(len(record)):
        for i in range(len(record[0][0])):
            record_result.append([j+1,record[j][0][i],record[j][1][i],record[j][2][i]])
    df = pd.DataFrame(data=record_result, columns=['k-fold','lda_svm_predict_value','label','task' ])
    
    
    
    df1= pd.DataFrame( columns=['close_eye_mouse_num','close_eye_mouse(correct_num)',
                                  'close_eye_nose_num','close_eye_nose(correct_num)',
                                  'open_eye_mouse_num','open_eye_mouse(correct_num)',
                                  'open_eye_nose_num','open_eye_nose(correct_num)',
                                  '1back_mouse_num','1back_mouse(correct_num)',
                                  '1back_nose_num','1back_nose(correct_num)',
                                  '2back_mouse_num','2back_mouse(correct_num)',
                                 '2back_nose_num','2back_nose(correct_num)']) 
    
    for i in range(1,11):
        df1.loc[i] = [
            len(df[df['k-fold']==i][df['task']=='rest_close_m']),len(df[df['k-fold']==i][df['task']=='rest_close_m'][df['lda_svm_predict_value']=='0']),
            len(df[df['k-fold']==i][df['task']=='rest_close_n']),len(df[df['k-fold']==i][df['task']=='rest_close_n'][df['lda_svm_predict_value']=='1']),
            len(df[df['k-fold']==i][df['task']=='rest_open_m']),len(df[df['k-fold']==i][df['task']=='rest_open_m'][df['lda_svm_predict_value']=='0']),
            len(df[df['k-fold']==i][df['task']=='rest_open_n']),len(df[df['k-fold']==i][df['task']=='rest_open_n'][df['lda_svm_predict_value']=='1']),

            len(df[df['k-fold']==i][df['task']=='1back_m']),len(df[df['k-fold']==i][df['task']=='1back_m'][df['lda_svm_predict_value']=='0']),
            len(df[df['k-fold']==i][df['task']=='1back_n']),len(df[df['k-fold']==i][df['task']=='1back_n'][df['lda_svm_predict_value']=='1']),

            len(df[df['k-fold']==i][df['task']=='2back_m']),len(df[df['k-fold']==i][df['task']=='2back_m'][df['lda_svm_predict_value']=='0']),
            len(df[df['k-fold']==i][df['task']=='2back_n']),len(df[df['k-fold']==i][df['task']=='2back_n'][df['lda_svm_predict_value']=='1'])
        ]

    

    
    return df , df1
                

In [242]:
train_set = 'total_m_n'

In [243]:
a,b=test(train_set ,k_parameter = 10,num_iter = 1)

svm best parameters
{'C': 7.000030517578125, 'gamma': 9.03125, 'kernel': 'rbf'}
['1back_m' '1back_m' '1back_n' '1back_n' '2back_m' '2back_m' '2back_n'
 '2back_n' 'rest_close_m' 'rest_close_m' 'rest_close_n' 'rest_close_n'
 'rest_open_m' 'rest_open_m' 'rest_open_n' 'rest_open_n']
['1back_m' '1back_m' '1back_n' '1back_n' '2back_m' '2back_m' '2back_n'
 '2back_n' 'rest_close_m' 'rest_close_m' 'rest_close_n' 'rest_close_n'
 'rest_open_m' 'rest_open_m' 'rest_open_n' 'rest_open_n']
['1back_m' '1back_m' '1back_n' '1back_n' '2back_m' '2back_m' '2back_n'
 '2back_n' 'rest_close_m' 'rest_close_m' 'rest_close_n' 'rest_close_n'
 'rest_open_m' 'rest_open_m' 'rest_open_n' 'rest_open_n']
['1back_m' '1back_m' '1back_n' '1back_n' '2back_m' '2back_m' '2back_n'
 '2back_n' 'rest_close_m' 'rest_close_m' 'rest_close_n' 'rest_close_n'
 'rest_open_m' 'rest_open_m' 'rest_open_n' 'rest_open_n']
['1back_m' '1back_m' '1back_n' '1back_n' '2back_m' '2back_m' '2back_n'
 '2back_n' 'rest_close_m' 'rest_close_m' 'rest_cl

In [247]:
b.to_csv('result.csv', mode='w')